# Field extentions

I think it's worth while to go over how pairing works in reality, since `MMM` doesn't do a practical dive into this, short of saying that $\mathbb{G}_2\subset \mathbb{F}_{p^{12}}$ in our case.

## Embedding degree

Why are we dealing with $\mathbb{F}_{p^{12}}$ at all? Well, we're technically dealing with not the entire EC group, but the cyclic subgroup $\langle X\rangle$ generated by the point $X$ which is the base of our DL problem (ie $X^d\equiv Y\implies [d]X = Y$). The embedding process (aka taking points from $\mathbb{G}_1$ and map them into $\mathbb{F}_{p^m}$) is done by the pairing $e(x, y)$ which, for a point $x$ in the n-order subgroup of the curve, will be an n-th root of unity for some $y$, required by the condition that $e(ax, by)=e(x,y)^{ab}$. In order for this to hold, there must be enough roots of unity in the field, which happens when $p^k\equiv 1\mod \ell$, where $\ell$ is the order of the cyclic subgroup. For us, this is $k=12$, so the embedding must go from the curve to $\mathbb{F}_{p^{12}}$. 

We need to deal with this massive extension for the pairing operation because the curve defined over this extension is the smallest extension which contains subgroups of order $r$ that we can use for pairings, one subgroup in which contains only points with zero trace, which we choose to be $\mathbb{G}_2$.

So we have $\mathbb{G}_1\subset E(\mathbb{F}_p)$ with $|\mathbb{G}_1|=r$, and $\mathbb{G}_2\subset E(\mathbb{F}_{p^{12}})$ with $|\mathbb{G}_2|=r$ which we want to use for our pairing.

## $\mathbb{F}_{p^{12}}$

Given an irreducible polynomial $N \in \mathbb{F}_p[x]$ of degree $m=12$, the elements of this extension are those given by $\{a_{m-1}x^{m-1}+\cdots+a_1x+a_0 \,|\, a_i\in \mathbb{F}_p\}$

Multiplication is defined by multiplying the two polynomials, then using polynomial long division on the polynomial $N$ to get the remainder, and inverses are defined via the extended euclidean algorithm. 

You can "tower" extensions if the order of one divides the order of the other, so if $m_j | m_{j+1}$, then $\mathbb{F}_p \subset \mathbb{F}_{p^{m_1}}\subset\cdots\subset \mathbb{F}_{p^{m_k}}$.

The [standard tower](https://eprint.iacr.org/2010/354.pdf) for BN254 is given by the following (see [here](https://github.com/ethereum/py_pairing/blob/master/py_ecc/bn128/bn128_field_elements.py) or [here](https://github.com/arkworks-rs/algebra/tree/master/curves/bn254/src/fields)):
\begin{align}
\mathbb{F}_{p^2}&=\mathbb{F}_{p}[u]/(u^2-\beta)\\
\mathbb{F}_{p^6}&=\mathbb{F}_{p^2}[v]/(v^3-\xi)\\
\mathbb{F}_{p^{12}}&=\mathbb{F}_{p^6}[w]/(w^2-v)
\end{align}
where $\beta$ is a quadratic nonresidue in $\mathbb{F}_p$ and $\xi$ neither a quadratic or cubic residue in $\mathbb{F_{p^2}}$, which amounts to saying that $X^6-\xi$ is irreducible in the ring $\mathbb{F}_{p^2}[X]$. Here, $\beta=-1,\xi=9+u$, which brings about $u^2=-1, w^2=v, v^3=9+u$, and therefore:
$$\mathbb{F}_{p^{12}} = \mathbb{F}_{p^2}[w]/(w^6-(9+u))$$

This brings about the following nice points
- any element in this extension can be written as $g+hw$ with $g,h\in\mathbb{F}_{p^6}$, which means that the $p^6$-th power of any element in the extension $x^{p^6}=g-hw$ is free to compute
- Likewise writing each $g,h$ in terms of coefficients from $\mathbb{F}_{p^2}$ lets you compute the $p$-th, $p^2$-th, and $p^3$-th powers easily as well

## Twists

Dealing with elements directly in $\mathbb{F}_{p^{12}}$ is very unruly and inefficient, but it is possible to define a coordinate transformation such that such that the curve in the 12-th order extension is mapped to a lower degree field. 

For BN254, we define a sextic twist (aka drops the degree of extension by 6) such that the twisted curve is defined on $\mathbb{F}_{p^2}$ instead of $\mathbb{F}_{p^{12}}$. Defining $u^6=(1+i)^{-1}$, the twist performs $(x,y)\to(x/u^2,y/u^3)$ to produce our new curve $E^\prime(\mathbb{F}_{p^2})$:

$$ y'^2 = x'^3 + \frac{3}{9+i}$$

Very nice. Note though that points in $E(\mathbb{F}_p)$ are pairs of ints, while points on the twist are pairs of complex ints, so points in $\mathbb{G}_2$ take more storage despite them being also valid as the domain for keys and signatures.

See [this](https://eips.ethereum.org/EIPS/eip-197) for industry definition of this twist.

Since $X^6-\xi$ is irreducible, with roots $w\in\mathbb{F}_{p^{12}}$, we therefore have a homomorphism
$$\Psi:E^\prime(\mathbb{F}_{p^2})\to E(\mathbb{F}_{p^{12}})\,;\,(x',y') = (w^2x', w^3y')$$

which is injective, but not surjective, and defines the twist mapping!

---

We also need one more thing before we talk about pairings. This is a bit technical (I know right, something in this document being technical? pshhaw), and is closely related to concepts from Galois theory which is above the scope of this review, but suffice to say you can define an *algrebraic closure* $\overline{\mathbb{F}}_p$, which is the field where every non-constant polynomial with coefficients in the field has a root in the field, which we can think about as the union of all valid extensions $\mathbb{F}_{p^m}$. Defining a curve over the algebraic closure is a concise way to say that we're interested in points lying in $\mathbb{F}_p, \mathbb{F}_{p^2}, \mathbb{F}_{p^6}, \mathbb{F}_{p^{12}}$ that satisfy the curve equation. 

There is a mapping $\phi_p:E(\overline{\mathbb{F}}_p)\to E(\overline{\mathbb{F}}_p); (x,y)\to(x^p,y^p)$ called the Frobenius morphism. It can [can be shown](https://link.springer.com/book/10.1007/978-0-387-09494-6) that the set of points fixed by $\phi$ are *exactly* the finite group $E(\mathbb{F}_p)$, so application of this mapping to the curve defined on the base field will leave things structurally unchanged. This mapping will crop back up later in our definition of optimal ate pairing.

## G1, G2

Now actually having the field extension primer, we can talk about what exactly $\mathbb{G}_1,\mathbb{G}_2$ are.

Recall that the $r$-torsion points of a curve are all the points $X$ such that $rX=\mathcal{O}$, with $\mathcal{O}$ the point at infinity. Ie, these are all points of order dividing r. We therefore define
- $\mathbb{G}_1\triangleq E(\mathbb{F}_p)[r]$ is the only subgroup of the $r$-torsion of $E$ on $\mathbb{F}_p$ of order $r$
- $\mathbb{G}_2\triangleq E^\prime(\mathbb{F}_{p^2})[r]$ is the only subgroup of the $r$-torsion of $E^\prime$ on $\mathbb{F}_{p^2}$ of order $r$

### Membership checks
#### G1

In our scheme, to hash a message to $E$, we use `hash_to_field` and `field_to_curve`, and then multiply the mapped curve point by the generator of the curve to create a point in $\mathbb{G}_1$. Fortunately, by [Theorem 2.3.1 of Silverman](https://link.springer.com/book/10.1007/978-0-387-09494-6), we have 
$$ |E(\mathbb{F}_p)|=p+1-t$$
and for BN curves generated by a value $z=2^{62}-2^{54}+2^{44}$, we have $p(z)+1-t(z)=r(z)$, implying that $|E(\mathbb{F}_p)|=r\implies \mathbb{G}_1 = E(\mathbb{F}_p)[r]=E(\mathbb{F}_p)$!

We therefore only need to check if a pair $(x,y)\in\mathbb{F}_r\times\mathbb{F}_r$ is on the curve $E(\mathbb{F}_p)$ for membership in $\mathbb{G}_1$. 

#### G2

This is a bit trickier. You can check easily if the point $(x,y)\in\mathbb{F}_{p^2}\times\mathbb{F}_{p^2}$ lies on $E^\prime(\mathbb{F}_{p^2})$, but unfortunately the order of the twist curve is not given by the order of the $r$-torsion, ie $|E^\prime(\mathbb{F}_{p^2})|=c_2r$, where $c_2$ is the $\mathbb{G}_2$ cofactor. [You can show](https://hackmd.io/@jpw/bn254#mathbb-G_2-order) that $c_2=p+t-1$. 

You can just rely on the definition of the $r$-torsion if you want to check if $[r](x,y)=\mathcal{O}$, but with 254 bits of r, this is slooooooow.

Faster algorithms exist. For instance, defined the untwist-Frobenius-twist endomorphism of [Galbraith-Scott](https://eprint.iacr.org/2008/117.pdf):

$$\psi:E^\prime(\mathbb{F}_{p^2})\to E^\prime(\mathbb{F}_{p^2}) = \Psi^{-1}\circ\phi_p\circ \Psi\,;\, (x^\prime, y^\prime)\to (\xi^{(p-1)/3}x^{\prime p}, \xi^{(p-1)/2}y^{\prime p})$$

where $\Psi$ is the twist mapping, and recall $\xi=9+u$. Membership in $\mathbb{G}_2$ therefore [boils down to verifying](https://eprint.iacr.org/2022/352.pdf) if the following holds: $Q=(x^\prime, y^\prime); \psi(Q)=[6x^2]Q$, and [more recent work](https://eprint.iacr.org/2022/348.pdf) improves this to the following:
$$[x+1]Q + \psi([x]Q) + \psi([x]Q) = \psi^3([2x]Q)$$

## optimal ate pairing

finally, we're here!! fuck.

So our goal here is to take a point $X\in\mathbb{G}_1= E(\mathbb{F}_p)$, and a point $Y\in\mathbb{G}_2\subset E^\prime(\mathbb{F}_{p^{12}})$, and map them to a point in a target group $\mathbb{G}_T\subset\mathbb{F}_{p^{12}}$, denoted by the map $e$, and corresponds qualitatively to multiplying a point in $\mathbb{G}_1$ by a point in $\mathbb{G}_2$. 

We need bilinearity, therefore requiring:
$$
e([a]X, [b]Y) = e(X, [b]Y)^a = e(X, Y)^{ab} = e(X, [a]Y)^b = e([b]X, [a]Y)
$$

The "best" way to create this $e$ is the "optimal ate pairing", which has an *excellent* guide for [high speed calculations in software](https://eprint.iacr.org/2010/354.pdf).

Before we dig into the pairing itself, we need to know how to define a line passing through two points on the twisted curve, and what the line is evaluated at a point on the curve. Specifcally, $R_1=(x^\prime_1, y^\prime_1), R_2=(x^\prime_2, y^\prime_2)\in E^\prime(\mathbb{F}_{p^2})$, and $T=(x,y)\in E(\mathbb{F}_p)$, we have the line $\ell$ defined as:

$$ 
\ell_{\Psi(R_1),\Psi(R_2)}(T) = \begin{cases}
w^2 (x^\prime_2-x^\prime_1)y + w^3(y^\prime_1-y^\prime_2)x + w^5(x^\prime_1 y^\prime_2-x^\prime_2 y^\prime_1) & R_1\neq R_2\\
(3x^{\prime 3}-2y^{\prime 2})(9+u) + w^3(2yy^\prime) + w^4(-3xx^{\prime 2})               & R_1=R_2
\end{cases}
$$

Armed with this knowledge, we now can define the optimal ate pairing $e:\mathbb{G}_1\times\mathbb{G}_2\to\mathbb{G}_T$ to be:

\begin{align}
e(X, Y) =\bigg(&f_{6z+2,Y}(X)\\
\times &\ell_{[6z+2]\Psi(Y),\phi_p(\Psi(Y))}(X)\\
\times &\ell_{[6z+2]\Psi(Y)+\phi_p(\Psi(Y)), -\phi_p(\Psi(Y))}(X)\bigg)^{\frac{p^{12}-1}{r}}
\end{align}

now THATS a mouthful, say that 5 times fast. Here, $z$ is the parameter of the curve. The pairing is based on rational functions $f_{i,Q}:\mathbb{N}\times\mathbb{G}_2\to\mathbb{F}_{p^{12}}$ that are evaluated iteratively in what's called Miller's algorithm. 

Fantastically, the [paper](https://crypto.stanford.edu/miller/miller.pdf) that describes this process was never published, but the algorithm, the imeplementation of which is refered to as "Miller's loops", says that:

$$
f_{i+j,Y} = f_{i,Y}f_{j,Y}\ell_{[i]\Psi(Y),[j]\Psi(Y)}
$$

TECHNICALLY there is another factor in the denominator of these iterations that describes the evaluation of the point $\Psi(Y)$ on the vertical line passing through $X$. However, we can ignore this evaluation, for reasons summarized well by [this](https://crypto.stanford.edu/pbc/notes/ep/optimize.html):

<blockquote>

To compute a Tate pairing, a quotient is iteratively calculated (Miller’s algorithm) and then raised to power of $(p^k-1)/r$, the Tate exponent. Each factor of the denominator is the equation of a vertical line evaluated at a particular point, i.e. the equation $X-a$ evaluated at some point $(x,y)$, which gives the factor $(x-a)$. 

Because of the way we have selected our groups, $x\in\mathbb{F}_{p^d}$, (note that the map $\Psi$ leaves the $x$-coordinate of its input in the same field), and $a\in\mathbb{F}_p$, hence $(x-a)\in\mathbb{F}_{p^d}$. 

Any element $a\in\mathbb{F}_{p^d}$ satisfies $a^{p^d-1}$. Observe $p^d-1$ divides $(p^k-1)/r$, because $r$ cannot divide $p^d-1$ (otherwise $d$ would be the embedding degree, not $k$). Thus each factor $(x-a)$ raised to the Tate exponent is 1, so it can be left out of the quotient. Hence, there is no need to compute the denominator at any time in Miller's algorithm.

</blockquote>

Slick.

### Toy implementation

In decimals, we know $z=4965661367192848881$, and therefore $6z+2=29793968203157093288$. Optimised implementations represent this bound in $\{-1, 0, 1\}$ basis, not binary, since it has a lower Hamming weight, just fyi, so in that case we get the following.

There will be miller's loop to determine the first term in the optimal ate pairing. Then for the final two terms, we notice that:

- $\ell_{[6z+2]\Psi(Y),\phi_p(\Psi(Y))}(X)$
    - Notice that $$\begin{align}\phi_p(\Psi(Y)) &= \left((w^2 x^\prime)^p, (w^3y^\prime)^p\right) \\&= \left(w^2\xi^{(p-1)/3}x^{\prime p}, w^3\xi^{(p-1)/2}y^{\prime p}\right) \\&= \Psi\left(\xi^{(p-1)/3}\bar{x}^\prime, \xi^{(p-1)/2}\bar{y}^\prime\right)\end{align}$$
    - Since $[n]\Psi(Q)=\Psi([n]Q)$ by the homomorphism, we just evaluate the line now at the point $Q^\prime = (\xi^{(p-1)/3}\bar{x}^\prime, \xi^{(p-1)/2}\bar{y}^\prime)=(x_1, y_1)$
    
- $\ell_{[6z+2]\Psi(Y)+\phi_p(\Psi(Y)), -\phi_p(\Psi(Y))}(X)$
    - You can likewise show that this is easily evaluated at the point $-Q$


In [ ]:
fn e(p: &G1Affine, q: &G2Affine) -> Fq12 {
    //membership checks, see sections above
    assert!(p.is_in_g1());
    assert!(q.is_in_g2());
    
    if p.is_identity().into() || q.is_identity.into() {
        return Fq12::One();
    }
    
    let mut r = *q;
    let mut f = Fq12::One(); //starting point of iteration
    
    //begin miller's loop, calculating f_{[6z+2],q}(p)
    for i in (0..BOUND.len() - 1).rev() {
        f = f * f * line(twist(&r), twist(&r), p);
        r = r.double();
        match BOUND[i] {
            1 => {
                f = f * line(untwist(&r), untwist(q), p);
                r.add_assign(q);
            },
            -1 => {
                f = f * line(untwist(&r),untwist(-q), p);
                r.sub_assign(q);
            },
            0 => {},
            _ => panic!("digit not in correct basis")
        }
    }
    let qp = q.frobenius_map();
    f = f * line(twist(&r), twist(&qp), p);
    r.add_assign(qp);
    let qpp = -qp.frobenius_map();
    f = f* line(twist(&r), twist(&qpp), p);
    
    final_exponentiation(&f)
}